<a href="https://colab.research.google.com/github/rohitrrg/sms-spam-classifier/blob/master/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('SMSSpamCollection', sep='\t', 
                 names=['label','message'])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.shape

(5572, 2)

In [ ]:
X = list(df['message'])
y = list(df['label'])

In [ ]:
y = list(pd.get_dummies(y, drop_first=True)['spam'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
import transformers
transformers.__version__

'4.25.1'

In [ ]:
  from transformers import AutoTokenizer

  tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(x_train, return_tensors="np", padding=True)
test_encodings = tokenizer(x_test, return_tensors="np", padding=True) 

In [ ]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data)

In [ ]:
import tensorflow as tf

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
train_dataset

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(238,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(238,), dtype=tf.int64, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_39', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
model.compile(optimizer=Adam(3e-5))

model.fit(train_dataset, epochs=2)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Epoch 1/2
4457/4457 [==============================] - 207s 45ms/step - loss: 0.0806
Epoch 2/2
4457/4457 [==============================] - 204s 46ms/step - loss: 0.0621


In [ ]:
model.evaluate(test_dataset)

1115/1115 [==============================] - 12s 10ms/step - loss: 0.0585


0.05849859118461609

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
))

In [ ]:
y_pred = model.predict(val_dataset)

In [ ]:
y_pred = np.argmax(y_pred[0], axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.9883408071748879
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       950
           1       0.97      0.95      0.96       165

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115

